In [14]:
https://www.linkedin.com/pulse/inputting-data-google-search-using-selenium-kartikey-garg

AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


import time

# Remplacez le chemin vers le driver de votre navigateur
driver_path = 'C:\chromedriver_win32\chromedriver.exe'
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)

# Ouvrir Google Maps
driver.get('https://www.google.com/maps/')

# Chercher le lieu
skip = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button/span').click()
search_box = driver.find_element(By.XPATH,'//*[@id="searchboxinput"]')
# Remplacer le lieu par votre lieu de recherche
test = ('Paris', 'New York')

for i in test:
    search_box.send_keys(i)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)
    search_box.clear()
    




print("The input Element is: ", search_box)



The input Element is:  <selenium.webdriver.remote.webelement.WebElement (session="14c0c1a950d1ed38135d857f17cc597c", element="716DEEE82991E39F4B6ECEB830F8F754_element_22")>


In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import dateparser

import time

# Remplacez le chemin vers le driver de votre navigateur
driver_path = 'chromedriver'
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)

# Ouvrir Google Maps
driver.get('https://www.google.com/maps/')

# Chercher le lieu
skip = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button/span').click()
search_box = driver.find_element(By.XPATH,'//*[@id="searchboxinput"]')
# Remplacer le lieu par votre lieu de recherche

search_box.send_keys('Centre des impots de Paris 16')
search_box.send_keys(Keys.ENTER)
time.sleep(2)
search_box.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]').click()
time.sleep(2)
text = search_box.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[4]/div/div/div[4]/div[1]/span[2]')
print(text.text)


    

7 months ago


In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import dateparser

import time

class GoogleMaps:
    def __init__(self):
        # Remplacez le chemin vers le driver de votre navigateur
        driver_path = 'driver/chromedriver'
        service = Service(executable_path=driver_path)
        self.driver = webdriver.Chrome(service=service)

    def search_location(self, location):
        # Ouvrir Google Maps
        self.driver.get('https://www.google.com/maps/')
        skip = self.driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[1]/div/div/button/span').click()
        search_box = self.driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
        search_box.send_keys(location)
        search_box.send_keys(Keys.ENTER)
        time.sleep(2)
        search_box.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]').click()
        time.sleep(2)
        text = search_box.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[4]/div/div/div[4]/div[1]/span[2]')
        return text.text

# Exemple d'utilisation
maps = GoogleMaps()
location = 'Centre des impots de Paris 16'
print(location)
result = maps.search_location(location)
print(result)





Centre des impots de Paris 16
7 months ago


In [10]:
import csv
import pandas as pd
reader = pd.read_csv('Data/export-structures.csv', sep=';')
    # itère à travers les lignes du fichier CSV
    for row in reader:
        # récupère toutes les valeurs de la 4ème colonne
        values = row[3].split(';')
        # itère à travers les valeurs de la 4ème colonne
        for location in values:
            # exécute la recherche de lieu en utilisant l'API Google Maps
            result = maps.search_location(location)
            # affiche le résultat de la recherche
            print(result)

IndexError: list index out of range

In [28]:
import csv
import pandas as pd


In [71]:
reader = pd.read_csv('Data/export-structures.csv', sep=';')

In [72]:
reader.shape

(14763, 26)

In [50]:
for row in reader:
    print(row)

"Code SP+ de la structure"
Intitulé du réseau
Intitulé de la typologie
Intitulé de la structure
Intitulé structure usager
Description de la structure
Identifiant de la structure
Type d'identifiant
Code SP+ structure parent
Intitulé de la structure parente
Adresse
Compléments Adresse
Code postal de la structure
Ville
PAYS
Codes Postaux
Départements
URL du site internet
Boîte à lettre fonctionnelle
Clé de tracking
Actif
Code sp+ de la typologie
INSEE Ville
INSEE Département
INSEE Région
INSEE Arrondissement


In [61]:
import pandas as pd

# lit le fichier CSV
df = pd.read_csv('Data/export-structures.csv', sep=';')

# sélectionne les lignes où la colonne "Actif" est "Oui"
df = df.loc[df["Actif"] == "Oui"]

# affiche le résultat
df


,"﻿""Code SP+ de la structure""",Intitulé du réseau,Intitulé de la typologie,Intitulé de la structure,Intitulé structure usager,Description de la structure,Identifiant de la structure,Type d'identifiant,Code SP+ structure parent,Intitulé de la structure parente,...,Départements,URL du site internet,Boîte à lettre fonctionnelle,Clé de tracking,Actif,Code sp+ de la typologie,INSEE Ville,INSEE Département,INSEE Région,INSEE Arrondissement
0,3Tn8gzTdcz,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DE LA MAYENNE,NaN,NaN,NaN,NaN,PB7y8Rc7w1,DZSP ZONE OUEST,...,NaN,NaN,NaN,NaN,Oui,amHofSPdoN,com53130,dep53,reg52,arr53130
1,GYL05KOxWg,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DE LA SEINE MARITIME,NaN,NaN,NaN,NaN,PB7y8Rc7w1,DZSP ZONE OUEST,...,NaN,NaN,NaN,NaN,Oui,amHofSPdoN,com76540,dep76,reg28,arr76540
2,PANEFF4wah,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES ALPES DE HAUTE PROVENCE,NaN,NaN,NaN,NaN,TPBhwmK7Sp,DZSP ZONE SUD,...,NaN,NaN,NaN,NaN,Oui,amHofSPdoN,com04070,dep04,reg93,arr04070
3,7VV7KUkiJY,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES BOUCHES DU RHONE,NaN,NaN,NaN,NaN,TPBhwmK7Sp,DZSP ZONE SUD,...,NaN,NaN,NaN,NaN,Oui,amHofSPdoN,com13055,dep13,reg93,arr13055-02
4,r7qjlPvWjQ,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES YVELINES,NaN,NaN,NaN,NaN,js724N8g2l,SP ILE DE FRANCE,...,NaN,NaN,NaN,NaN,Oui,amHofSPdoN,com78646,dep78,reg11,arr78646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14758,STR-167,Ministère de la Culture,Monuments nationaux,Wimille-Colonne de la Grande Armée,NaN,Monument,NaN,NaN,STR-94,Centre des monuments nationaux,...,NaN,NaN,NaN,NaN,Oui,TYP-17,com62894,dep62,reg32,arr62894
14759,STR-1244,France services,Structure France services,Wissembourg,France services de Wissembourg,Structure labellisée France services sur le te...,2488,Matricule,caXOde9y4y,ANCT,...,NaN,NaN,NaN,NaN,Oui,rOVlkUEZBP,com67544,dep67,reg44,arr67544
14760,kinnEFK8ZI,POLE EMPLOI,AGENCE POLE EMPLOI (APE),WISSEMBOURG-PE,NaN,WISSEMBOURG-PE,GRE0192,NaN,u62r57ufM8,DTD CENTRE ET NORD ALSACE-PE,...,NaN,NaN,NaN,NaN,Oui,VBk7hCR4zw,com67544,dep67,reg44,arr67544
14761,STR-1389,France services,Structure France services,Wittelsheim,France services de Wittelsheim,Structure labellisée France services sur le te...,2648,Matricule,caXOde9y4y,ANCT,...,NaN,NaN,NaN,NaN,Oui,rOVlkUEZBP,com68375,dep68,reg44,arr68375


In [170]:
import pandas as pd
import numpy as np

# lit le fichier CSV
df = pd.read_csv('Data/export-structures.csv', sep=';')
df["GM_search"] = df[["Adresse", "Intitulé du réseau","Intitulé de la typologie", "Intitulé de la structure", "Ville"]].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
df1 = df[["Adresse", "Intitulé du réseau","Intitulé de la typologie", "Ville", "Intitulé de la structure", "GM_search"]]
df1 = df1.sample(n=1, replace=False, random_state=np.random.randint(100))
df1


,Adresse,Intitulé du réseau,Intitulé de la typologie,Ville,Intitulé de la structure,GM_search
5188,144 Rue de Bavay,Réseau des rectorats et DSDEN,Rectorat,LILLE,RECTORAT DE L'ACADEMIE DE LILLE,"144 Rue de Bavay, Réseau des rectorats et DSDE..."


In [269]:
df_row = df.loc[5063]
df_row

"Code SP+ de la structure"                                              4IPD5w6S7f
Intitulé du réseau                                                            DGFIP
Intitulé de la typologie                                           Trésorerie mixte
Intitulé de la structure                                         Trésorerie CONTRES
Intitulé structure usager                                                       NaN
Description de la structure                                      Trésorerie CONTRES
Identifiant de la structure                                                 4109E03
Type d'identifiant                                                       code DGFIP
Code SP+ structure parent                                                fe6fzGrv89
Intitulé de la structure parente           DIR DEPT FINANCES PUBLIQUES LOIR-ET-CHER
Adresse                                                                         NaN
Compléments Adresse                                                          

In [79]:

for row in df.iterrows():
    print(row[1]['Nom et adresse'])

DDSP DE LA MAYENNE, 7 Place Mendes France
DDSP DE LA SEINE MARITIME, 9 Rue Brisout de Barneville
DDSP DES ALPES DE HAUTE PROVENCE, Boulevard Victor Hugo
DDSP DES BOUCHES DU RHONE, 2 Rue Antoine Becker
DDSP DES YVELINES, 19 Avenue de Paris
DDSP DU CHER, 6 Avenue d'Orléans
DDSP DU FINISTERE, 3 Rue Théodore le Hars
DDSP DU HAUT RHIN, 43 Rue de la Mertzau
DDSP DU JURA, 6 Avenue du 44 eme Régiment d' Infanterie
DDSP DU MAINE ET LOIRE, 15 bis Rue du Petit Thouars
DDSP DU NORD, 19 Rue des Marquillies
DDSP DU TERRITOIRE DE BELFORT, 1-3 Rue du Manège
DDSP DU VAL D'OISE, 4 Rue de la Croix des Maheux
DDSP DU VAR, 1 Rue du Commissaire Morandin
DEAL Martinique, Pointe de Jaham
DEAL Mayotte
Delme, 2 rue Maurice Vautrin
Département de la diffusions pluridisciplinaire et des programmes transverses
Derval, 14 rue de la Garlais
Relais d'accès au droit Maison Du Département de Limoges, 24 avenue du Président Coty
Relais d'accès au droit Maison rurale de Saint Maur, Bel Air
Relais d'accès au droit Mamers,

In [167]:
type(text_info)

list

In [214]:
import numpy as np
df1 = df.sample(n=1, replace=False, random_state=np.random.randint(100))
df1


,"﻿""Code SP+ de la structure""",Intitulé du réseau,Intitulé de la typologie,Intitulé de la structure,Intitulé structure usager,Description de la structure,Identifiant de la structure,Type d'identifiant,Code SP+ structure parent,Intitulé de la structure parente,...,URL du site internet,Boîte à lettre fonctionnelle,Clé de tracking,Actif,Code sp+ de la typologie,INSEE Ville,INSEE Département,INSEE Région,INSEE Arrondissement,Nom et adresse
11105,qiy8tx3s3N,France services,Structure France services,Association d'Animation du Beaufortain,France services Association d'Animation du Bea...,Structure labellisée France services sur le te...,1583,Matricule,caXOde9y4y,ANCT,...,NaN,NaN,NaN,Oui,rOVlkUEZBP,com73034,dep73,reg84,arr73034,"Association d'Animation du Beaufortain, 36, ru..."


In [224]:
df.columns

Index(['﻿"Code SP+ de la structure"', 'Intitulé du réseau',
       'Intitulé de la typologie', 'Intitulé de la structure',
       'Intitulé structure usager', 'Description de la structure',
       'Identifiant de la structure', 'Type d'identifiant',
       'Code SP+ structure parent', 'Intitulé de la structure parente',
       'Adresse', 'Compléments Adresse', 'Code postal de la structure',
       'Ville', 'PAYS', 'Codes Postaux', 'Départements',
       'URL du site internet', 'Boîte à lettre fonctionnelle',
       'Clé de tracking', 'Actif', 'Code sp+ de la typologie', 'INSEE Ville',
       'INSEE Département', 'INSEE Région', 'INSEE Arrondissement',
       'Nom et adresse'],
      dtype='object')

In [220]:
df 

,"﻿""Code SP+ de la structure""",Intitulé du réseau,Intitulé de la typologie,Intitulé de la structure,Intitulé structure usager,Description de la structure,Identifiant de la structure,Type d'identifiant,Code SP+ structure parent,Intitulé de la structure parente,...,URL du site internet,Boîte à lettre fonctionnelle,Clé de tracking,Actif,Code sp+ de la typologie,INSEE Ville,INSEE Département,INSEE Région,INSEE Arrondissement,Nom et adresse
0,3Tn8gzTdcz,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DE LA MAYENNE,NaN,NaN,NaN,NaN,PB7y8Rc7w1,DZSP ZONE OUEST,...,NaN,NaN,NaN,Oui,amHofSPdoN,com53130,dep53,reg52,arr53130,"DDSP DE LA MAYENNE, 7 Place Mendes France"
1,GYL05KOxWg,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DE LA SEINE MARITIME,NaN,NaN,NaN,NaN,PB7y8Rc7w1,DZSP ZONE OUEST,...,NaN,NaN,NaN,Oui,amHofSPdoN,com76540,dep76,reg28,arr76540,"DDSP DE LA SEINE MARITIME, 9 Rue Brisout de Ba..."
2,PANEFF4wah,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES ALPES DE HAUTE PROVENCE,NaN,NaN,NaN,NaN,TPBhwmK7Sp,DZSP ZONE SUD,...,NaN,NaN,NaN,Oui,amHofSPdoN,com04070,dep04,reg93,arr04070,"DDSP DES ALPES DE HAUTE PROVENCE, Boulevard Vi..."
3,7VV7KUkiJY,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES BOUCHES DU RHONE,NaN,NaN,NaN,NaN,TPBhwmK7Sp,DZSP ZONE SUD,...,NaN,NaN,NaN,Oui,amHofSPdoN,com13055,dep13,reg93,arr13055-02,"DDSP DES BOUCHES DU RHONE, 2 Rue Antoine Becker"
4,r7qjlPvWjQ,POLICE NATIONALE,DIRECTION DEPARTEMENTALE DE LA SECURITE PUBLIQUE,DDSP DES YVELINES,NaN,NaN,NaN,NaN,js724N8g2l,SP ILE DE FRANCE,...,NaN,NaN,NaN,Oui,amHofSPdoN,com78646,dep78,reg11,arr78646,"DDSP DES YVELINES, 19 Avenue de Paris"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14758,STR-167,Ministère de la Culture,Monuments nationaux,Wimille-Colonne de la Grande Armée,NaN,Monument,NaN,NaN,STR-94,Centre des monuments nationaux,...,NaN,NaN,NaN,Oui,TYP-17,com62894,dep62,reg32,arr62894,Wimille-Colonne de la Grande Armée
14759,STR-1244,France services,Structure France services,Wissembourg,France services de Wissembourg,Structure labellisée France services sur le te...,2488,Matricule,caXOde9y4y,ANCT,...,NaN,NaN,NaN,Oui,rOVlkUEZBP,com67544,dep67,reg44,arr67544,"Wissembourg, 2 Rue du tribunal"
14760,kinnEFK8ZI,POLE EMPLOI,AGENCE POLE EMPLOI (APE),WISSEMBOURG-PE,NaN,WISSEMBOURG-PE,GRE0192,NaN,u62r57ufM8,DTD CENTRE ET NORD ALSACE-PE,...,NaN,NaN,NaN,Oui,VBk7hCR4zw,com67544,dep67,reg44,arr67544,WISSEMBOURG-PE
14761,STR-1389,France services,Structure France services,Wittelsheim,France services de Wittelsheim,Structure labellisée France services sur le te...,2648,Matricule,caXOde9y4y,ANCT,...,NaN,NaN,NaN,Oui,rOVlkUEZBP,com68375,dep68,reg44,arr68375,"Wittelsheim, 4 rue de Cernay"


In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14763 entries, 0 to 14762
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ﻿"Code SP+ de la structure"       14763 non-null  object 
 1   Intitulé du réseau                14763 non-null  object 
 2   Intitulé de la typologie          14763 non-null  object 
 3   Intitulé de la structure          14763 non-null  object 
 4   Intitulé structure usager         2237 non-null   object 
 5   Description de la structure       12752 non-null  object 
 6   Identifiant de la structure       9487 non-null   object 
 7   Type d'identifiant                7452 non-null   object 
 8   Code SP+ structure parent         14725 non-null  object 
 9   Intitulé de la structure parente  14725 non-null  object 
 10  Adresse                           7471 non-null   object 
 11  Compléments Adresse               73 non-null     object 
 12  Code

In [10]:
# créer une série temporelle
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# créer un modèle AR de première ordre
model = AutoRegressiveModel(p=1)

# estimer les coefficients d'autorégression à partir des données
model.fit(X)

# prédire les valeurs futures de la série temporelle
predictions = model.predict(X)
print(predictions)


[10. 10. 10. 10. 10. 10. 10. 10. 10.]


# Scaping avec bs4

In [4]:
import requests

from bs4 import BeautifulSoup


In [136]:
class TextInformation:
    def __init__(self, url):
        self.url = url
        response = requests.get(self.url)
        self.soup = BeautifulSoup(
            response.text,
            'html.parser',
            from_encoding='utf-8')

    def get_name(self):
        name = self.soup.select('div', {'class': 'm6QErb '})
        
            
url = 'https://www.google.com/maps/place/Centre+of+Public+Finance/@48.857849,2.359733,14z/data=!4m12!1m2!2m1!1scentre+des+impots!3m8!1s0x47e66df7cf592dd7:0x567a2849b9d5baa2!8m2!3d48.8563258!4d2.380233!9m1!1b1!15sChFjZW50cmUgZGVzIGltcG90c5IBD2NpdHlfdGF4X29mZmljZeABAA!16s%2Fg%2F1q67dg8mw'
TextInformation(url).get_name()


[<div class="signin"><a class="basebutton hairlinebutton" href="https://accounts.google.com/ServiceLogin?hl=fr&amp;continue=https://www.google.com/maps/place/Centre%2Bof%2BPublic%2BFinance/@48.857849,2.359733,14z/data%3D!4m12!1m2!2m1!1scentre%2Bdes%2Bimpots!3m8!1s0x47e66df7cf592dd7:0x567a2849b9d5baa2!8m2!3d48.8563258!4d2.380233!9m1!1b1!15sChFjZW50cmUgZGVzIGltcG90c5IBD2NpdHlfdGF4X29mZmljZeABAA!16s%252Fg%252F1q67dg8mw&amp;gae=cb-eomtm">Se connecter</a></div>, <div class="box"><img alt="Google" height="28" src="//www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_68x28dp.png" srcset="//www.gstatic.com/images/branding/googlelogo/2x/googlelogo_color_68x28dp.png 2x" width="68"/><h1>Avant d'accéder à Google</h1><div class="contentText">Nous utilisons des <a href="https://policies.google.com/technologies/cookies?hl=fr&amp;utm_source=ucb" target="_blank">cookies</a> et d'autres données pour :<ul><li>Proposer les services Google et s'assurer qu'ils fonctionnent correctement</li><li>S

/Users/camille/repo/Hetic/projet_gouv/Services-Publics/gouv_env/lib/python3.10/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [59]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

class TextInformation:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome() # Ajout d'un objet driver
        self.driver.get(url) # Ouverture de l'URL dans le navigateur
    def get_name(self):
        name = self.driver.find_element(By.XPATH,'//*[@id="title"]/span[1]/span[1]')
        return name.text.strip()

url = 'https://www.google.com/maps/place/Centre+of+Public+Finance/@48.857849,2.359733,14z/data=!4m12!1m2!2m1!1scentre+des+impots!3m8!1s0x47e66df7cf592dd7:0x567a2849b9d5baa2!8m2!3d48.8563258!4d2.380233!9m1!1b1!15sChFjZW50cmUgZGVzIGltcG90c5IBD2NpdHlfdGF4X29mZmljZeABAA!16s%2Fg%2F1q67dg8mw'
text_info = TextInformation(url)
print(text_info.get_name())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="title"]/span[1]/span[1]"}
  (Session info: chrome=113.0.5672.63)
Stacktrace:
0   chromedriver                        0x00000001024af8ac chromedriver + 4257964
1   chromedriver                        0x00000001024a7f40 chromedriver + 4226880
2   chromedriver                        0x00000001020e49d4 chromedriver + 281044
3   chromedriver                        0x000000010211fa34 chromedriver + 522804
4   chromedriver                        0x00000001021567e4 chromedriver + 747492
5   chromedriver                        0x000000010211398c chromedriver + 473484
6   chromedriver                        0x000000010211498c chromedriver + 477580
7   chromedriver                        0x000000010246e900 chromedriver + 3991808
8   chromedriver                        0x0000000102472354 chromedriver + 4006740
9   chromedriver                        0x0000000102472940 chromedriver + 4

In [ ]:
df1.to_csv('Data/rec.csv', index=False, sep=';')